## Imports

In [37]:
import pandas as pd
import os
import json
import re
import spacy
import nmslib

In [34]:
!pip install nmslib

    100% |████████████████████████████████| 256kB 11.8MB/s ta 0:00:01
    100% |████████████████████████████████| 153kB 42.3MB/s ta 0:00:01
  Running setup.py bdist_wheel for nmslib ... error
  Complete output from command /home/onepanel/.conda/bin/python -u -c "import setuptools, tokenize;__file__='/tmp/pip-install-aizvpy22/nmslib/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d /tmp/pip-wheel-pq7px92p --python-tag cp36:
  running bdist_wheel
  running build
  running build_ext
  creating tmp
  gcc -pthread -B /home/onepanel/.conda/compiler_compat -Wl,--sysroot=/ -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -I/home/onepanel/.conda/include/python3.6m -c /tmp/tmpu2b6c3od.cpp -o tmp/tmpu2b6c3od.o -std=c++14
  cc1plus: warning: command line option ‘-Wstrict-prototypes’ is valid for C/ObjC but not for C++
  gcc -pthread -B /home/onepanel/.conda/compiler_compat 

In [22]:
!python -m spacy download en

    100% |████████████████████████████████| 37.4MB 87.5MB/s ta 0:00:01
  Running setup.py install for en-core-web-sm ... done
You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

    Linking successful
    /home/onepanel/.conda/lib/python3.6/site-packages/en_core_web_sm -->
    /home/onepanel/.conda/lib/python3.6/site-packages/spacy/data/en

    You can now load the model via spacy.load('en')



In [2]:
!pip install xlrd

kaggle 1.5.0 has requirement urllib3<1.23.0,>=1.15, but you'll have urllib3 1.24.1 which is incompatible.
fastai 1.0.18 has requirement notebook>=5.7.0, but you'll have notebook 5.6.0 which is incompatible.
boto3 1.9.125 has requirement botocore<1.13.0,>=1.12.125, but you'll have botocore 1.12.93 which is incompatible.
You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


# Read xslx file sheet

In [16]:
data = pd.read_excel('../data/QA_(0-100).xlsx','Sheet1',  index_col=0, header=None)

In [17]:
data = data.iloc[:, 0:2].reset_index(drop=True)

In [19]:
data.columns = ['question', 'answer']

In [23]:
data.head(2)

,question,answer
0,Carl and the Passions changed band name to what,Beach Boys
1,How many rings on the Olympic flag,Five


# Create QA model class

In [70]:
class QA(object):
    def __init__(self, data):
        self.nlp = spacy.load('en')
        self.questions = data.question.tolist()
        self.answers = data.answer.tolist()
    
    def to_vectors(self, texts):
        """Convert texts into their vectors"""
        result = []
        for item in texts:
            result.append(self.nlp(item).vector)
        
        return result
            
    def build_nmslib_index(self):
        """build nmslib index with vectors of question texts"""
        self.index = {}
        self.index = nmslib.init(method='hnsw', space='cosinesimil')
        self.index.addDataPointBatch(self.to_vectors(self.questions))
        self.index.createIndex({'post': 2}, print_progress=True)
        
    def search(self, text, max_distance=0.2):
        """
        K-Nearest-Neighbour search over indexed taxonomy data and distance threshold parameter 
        to get most similar one. 
        Args:
            text: (str) sample question text
            max_distance: (float) maximum allowed distance for neighbours

        Returns:
            result: (tuple) index and distance for found item

        """
        result = {}
        vector = self.nlp(text).vector
        
        if vector is not None:
            ids, distances = self.index.knnQuery(vector)
            
            if ids is not None and distances is not None:
                best_indices_mask = (distances == distances.min()) & (distances < max_distance)
                if best_indices_mask.sum() != 0:
                    result = {'index': ids[best_indices_mask][0], 'distance': distances[best_indices_mask][0]}

        return result
    
    def query(self, question, max_distance=0.2):
        search_result = self.search(question, max_distance)
        index, distance = search_result.get('index', -1), search_result.get('distance', -1)
        result = "N/A"
        if index != -1:
            result = self.answers[index]
        
        return result

In [71]:
qa = QA(data)
qa.build_nmslib_index()

In [85]:
qa.query('Carl and the Passions day changed band name to what', max_distance=0.05)

'Beach Boys'

In [58]:
data.head(2)

,question,answer
0,Carl and the Passions changed band name to what,Beach Boys
1,How many rings on the Olympic flag,Five


In [59]:
preds = data.question.apply(lambda x: qa.query(x))